In [1]:
import sys 
sys.path.insert(0, '/home/yikuan/project/Code/')

In [2]:
from common.pytorch import save_model,load_model
from common.common import create_folder, load_obj
import os
import torch
from ACM.model.utils.utils import age_vocab
import pandas as pd
import pytorch_pretrained_bert as Bert
from ACM.dataLoader.HF import HF_data
from ACM.model.bertKISSGP import BertHF
import gpytorch
from ACM.model.optimiser import adam
import torch.nn as nn
from sklearn.metrics import average_precision_score, roc_auc_score

In [3]:
class BertConfig(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfig, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings=config.get('max_position_embeddings'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        self.num_labels = config.get('num_labels')


class TrainConfig(object):
    def __init__(self, config):
        self.batch_size = config.get('batch_size')
        self.use_cuda = config.get('use_cuda')
        self.max_len_seq = config.get('max_len_seq')
        self.train_loader_workers = config.get('train_loader_workers')
        self.test_loader_workers = config.get('test_loader_workers')
        self.device = config.get('device')
        self.output_dir = config.get('output_dir')
        self.output_name = config.get('output_name')
        self.best_name = config.get('best_name')

In [4]:
file_config = {
    'vocab':'/home/shared/yikuan/HF/Data/PureICD/Full_vocab',
    'train': '/home/shared/01_data/03_cuts/01_model_improvement/separate_med_diag/testV1BertDM.parquet',
    'test': '/home/shared/01_data/03_cuts/01_model_improvement/separate_med_diag/testV1BertDM.parquet'
}

In [5]:
global_params = {
    'max_seq_len': 256,
    'max_age': 110,
    'month': 1,
    'age_symbol': None,
}

optim_param = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

In [6]:
BertVocab = load_obj(file_config['vocab'])
ageVocab, _ = age_vocab(max_age=global_params['max_age'], mon=global_params['month'], symbol=global_params['age_symbol'])

In [7]:
trainData_raw = pd.read_parquet(file_config['train'])
testData_raw = pd.read_parquet(file_config['test'])

In [8]:
train_params = {
    'batch_size': 64,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'train_loader_workers': 3,
    'test_loader_workers': 3,
    'device': 'cuda:1',
    'output_dir': '/home/shared/yikuan/ACM/model/HF',
    'output_name': 'behrtKISSGPCalibration.bin',
    'best_name': 'behrtKISSGPCalibration_best.bin',
}

In [9]:
trainConfig = TrainConfig(train_params)

data_set = HF_data(trainConfig, trainData_raw, testData_raw, BertVocab['token2idx'], ageVocab, code='code', age='age')

trainData = data_set.get_weighted_sample_train(4)
testData = data_set.get_test_loader()

In [10]:
model_param = {
    'vocab_size': len(BertVocab['token2idx'].keys()),
    'hidden_size': 150,
    'num_hidden_layers': 4,
    'num_attention_heads': 6,
    'hidden_act': 'gelu',
    'intermediate_size': 108,
    'max_position_embeddings': global_params['max_seq_len'],
    'seg_vocab_size': 2,
    'age_vocab_size': len(ageVocab.keys()),
    'prior_prec': 1e0,
    'prec_init': 1e0,
    'initializer_range': 0.02,
    'num_labels': 1,
    'hidden_dropout_prob': 0.29,
    'attention_probs_dropout_prob': 0.38
}

In [11]:
modelConfig = BertConfig(model_param)

model = BertHF(modelConfig, n_dim=24,grid_size=40, ard_num_dims=24)

likelihood = gpytorch.likelihoods.BernoulliLikelihood().to(trainConfig.device)
mll = gpytorch.mlls.VariationalELBO(likelihood, model.gp_layer, num_data=len(trainData.dataset))

In [12]:
class Scaling(nn.Module):
    def __init__(self):
        super(Scaling, self).__init__()

        self.a = nn.Parameter(torch.randn(1))
        self.b = nn.Parameter(torch.randn(1))
    
    def forward(self, logits, labels=None):
        logits = logits.view(-1)
        logits = torch.mul(self.a, logits) + self.b
        
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels.view(-1))
            return loss
        else:
            return logits

In [13]:
# def load_model(path, model):
#     # load pretrained model and update weights
#     pretrained_dict = torch.load(path)
#     model_dict = model.state_dict()
#     # 1. filter out unnecessary keys
#     pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and k not in ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']}
#     # 2. overwrite entries in the existing state dict
#     model_dict.update(pretrained_dict)
#     # 3. load the new state dict
#     model.load_state_dict(model_dict)
#     return model

model = load_model('/home/shared/yikuan/Model/BayesianGPBERT/BayesianBERTGP_diag_med_5_best.bin', model)
model = model.to(trainConfig.device)

scaleHead = Scaling()
scaleHead = scaleHead.to(trainConfig.device)

optim = adam(list(scaleHead.named_parameters()),config=optim_param)

t_total value of -1 results in schedule not being applied


In [17]:
def precision(logits, label):
    sig = nn.Sigmoid()
    output=sig(logits)
    label, output=label.cpu(), output.detach().cpu()
    tempprc= average_precision_score(label.numpy(),output.numpy())
    auc = roc_auc_score(label.numpy(),output.numpy())
    return tempprc, output, label

def precision_test(logits, label):
    sig = nn.Sigmoid()
    output=sig(logits)
    tempprc= average_precision_score(label.cpu().numpy(),output.cpu().numpy())
    auc = roc_auc_score(label.cpu().numpy(),output.cpu().numpy())
    return tempprc, auc

In [15]:
def train(e, trainload, model, likelihood, optim, scaleHead):
    model.train()
    likelihood.train()

    tr_loss = 0
    temp_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    cnt = 0
    for step, batch in enumerate(trainload):
        cnt += 1

        age_ids, input_ids, posi_ids, segment_ids, attMask, targets, _, _ = batch

        age_ids = age_ids.to(train_params['device'])
        input_ids = input_ids.to(train_params['device'])
        posi_ids = posi_ids.to(train_params['device'])
        segment_ids = segment_ids.to(train_params['device'])
        attMask = attMask.to(train_params['device'])
        targets = targets.view(-1).to(train_params['device'])

        output = model(input_ids, age_ids, segment_ids, posi_ids, attention_mask=attMask, labels=targets)
#         loss = -mll(output, targets)
        loss = scaleHead(output.sample(), targets)
    
        loss.backward()

        temp_loss += loss.item()
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1

        if step % 500 == 0:
            #             prec, a, b = precision(logits, targets)
            #             print("epoch: {}\t| Cnt: {}\t| Loss: {}\t| precision: {}".format(e, cnt, temp_loss / 500, prec))
            print("epoch: {}\t|step: {}\t|Loss: {}".format(e, step, temp_loss / 500))
            temp_loss = 0

        optim.step()
        optim.zero_grad()

    # Save a trained model
    output_model_file = os.path.join(trainConfig.output_dir, trainConfig.output_name)
    create_folder(trainConfig.output_dir)
    save_model(output_model_file, scaleHead)


def evaluation(testload, model, likelihood, scaleHead):
    model.eval()
    y = []
    y_label = []
    loss_temp = 0

    model.eval()
    likelihood.eval()
    with torch.no_grad():
        for step, batch in enumerate(testload):
            age_ids, input_ids, posi_ids, segment_ids, attMask, targets, _, _ = batch

            age_ids = age_ids.to(train_params['device'])
            input_ids = input_ids.to(train_params['device'])
            posi_ids = posi_ids.to(train_params['device'])
            segment_ids = segment_ids.to(train_params['device'])
            attMask = attMask.to(train_params['device'])
            targets = targets.view(-1).to(train_params['device'])

            output = model(input_ids, age_ids, segment_ids, posi_ids, attention_mask=attMask, labels=targets)
            logits = scaleHead(output.sample())
            
#             logits = output.mean.float().cpu()
            targets = targets.cpu()

            y_label.append(targets)
            y.append(logits)

        y_label = torch.cat(y_label, dim=0)
        y = torch.cat(y, dim=0)

        tempprc, auc = precision_test(y.view(-1), y_label.view(-1))
        return tempprc, auc

In [18]:
best_pre = 0
for e in range(20):
    train(e, trainData, model, likelihood,optim,scaleHead)
#     auc_train, time_cost_train = evaluation(trainData, model)
    auc_test, auc = evaluation(testData, model, likelihood,scaleHead)
    print('test precision: {}, {}'.format(auc_test, auc))
    if auc_test >best_pre:
        # Save a trained model
        output_model_file = os.path.join(trainConfig.output_dir, trainConfig.best_name)
        create_folder(trainConfig.output_dir)
        save_model(output_model_file, scaleHead)
        best_pre = auc_test

epoch: 0	|step: 0	|Loss: 0.0023151030540466307
epoch: 0	|step: 500	|Loss: 1.0706603890657425
epoch: 0	|step: 1000	|Loss: 1.0508060804605484
epoch: 0	|step: 1500	|Loss: 1.0225660632848739
epoch: 0	|step: 2000	|Loss: 1.0049191238880157
epoch: 0	|step: 2500	|Loss: 0.9823333733081817
epoch: 0	|step: 3000	|Loss: 0.9639585037231445
epoch: 0	|step: 3500	|Loss: 0.9435020264387131
** ** * Saving fine - tuned model ** ** * 
test precision: 0.04648299156863731, 0.15138070256431088
** ** * Saving fine - tuned model ** ** * 
epoch: 1	|step: 0	|Loss: 0.0017506577968597413
epoch: 1	|step: 500	|Loss: 0.9130603817701339
epoch: 1	|step: 1000	|Loss: 0.8932200891971588
epoch: 1	|step: 1500	|Loss: 0.8757926768064499
epoch: 1	|step: 2000	|Loss: 0.8593227022886276
epoch: 1	|step: 2500	|Loss: 0.8382360107898712
epoch: 1	|step: 3000	|Loss: 0.8244403784275055
epoch: 1	|step: 3500	|Loss: 0.8070649275779724
** ** * Saving fine - tuned model ** ** * 
test precision: 0.0464849519878532, 0.15155014362394242
** ** * 

KeyboardInterrupt: 